**Notebook Autor: Souvik Ganguly**

Data source: https://huggingface.co/datasets/codesignal/lending-club-loan-accepted/tree/main <br>
Data stored in google drive: /content/drive/MyDrive/Lending_Club_Analysis/data/accepted_2007_to_2018Q4.csv <br>

# **This notebook generates:**<br>
**1. A DQ report for all the variables in the dataset**<br>
**2. 2 descriptive statistical reports for columns with missing values and columns with no missing values**<br>
**3. dataframe_for_eda.csv: Dataframe where columns with missing values >=40% are dropped.**<br>


# **DQ Report:**
**Generates a report with the below data points:**<br>
**1. Missing value %**<br>
**2. Number of Unique values**<br>
**3. Data Types**<br>
**4. Descriptive statistical measures**<br>
**5. Type of distribution**

# **Descriptive statistical reports:**
**This is an exhaustive analysis workbook where each variables having missing values are analyzed and below conclusions are drawn from the analysis:**<br>
**1. count**<br>
**2. mean**<br>
**3. median**<br>
**4. percentile values (0th %ile, 25th %ile, 50th %ile, 75th %ile, 100th %ile)**<br>
**5. min**<br>
**6. max**<br>
<br>


**Supporting .py files with function scripts to generate above outputs:**<br>
  **1. dq_report.py**<br>
  **2. analysis_functions_columns.py**


# **1. Load python packages and dataset**

In [ ]:
# LIBRARIES -----
!pip install xlsxwriter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Set seaborn style
sns.set(style="whitegrid")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
#Mount google drive folder
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#read the transactions data
df = pd.read_csv('/content/drive/MyDrive/Lending_Club_Analysis/data/accepted_2007_to_2018Q4.csv', low_memory=False)

In [ ]:
df.shape

(2260701, 151)

In [ ]:
#First 5 rows of data
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,N

As part of this analysis we're interested to dive deep into the loan records that are either Fully Paid or is charged off.

In [ ]:
#excluding rows with loan_status other than 'Fully Paid' and 'Charged Off'
df=df.loc[df['loan_status'].isin(['Fully Paid', 'Charged Off'])]

In [ ]:
df.shape

(1345310, 151)

# **2.0 Generate an exhaustive Data Quality Report**

In [ ]:
from google.colab import files
files.upload()  # This will prompt you to upload the dq_report.py file from your local machine

import shutil

# Assuming the dq_report.py file is uploaded to /content directory
shutil.move("dq_report.py", "/content/dq_report.py")

Saving dq_report.py to dq_report.py


'/content/dq_report.py'

In [ ]:
# Import custom functions from dq_report.py
from dq_report import data_quality_report, export_report_to_excel

# Generate the data quality report
report = data_quality_report(df)

# Export the report to an Excel file
export_report_to_excel(report)

/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


Data quality report exported successfully to data_quality_report.xlsx


# **Summary of DQ report:**

**Numerical and categorical variables:**<br>
113 numerical variables<br>
38 categorical variables

**Null values distributions**<br>
Number of columns with null values > 40% : 58<br>
Number of columns with null values > 10% and null values < 40% : 1 i.e;mths_since_recent_inq<br>
Number of columns with null values > 0% and null values < 10% : 40<br>
Column with no null values: id
Number of columns with no null values: 52



# **3.0 Generate the descriptive statistical report of rest of the columns**

**Based on the data dictionary and discussion with stakeholders it has been agreed that columns with null values > 40% should be dropped**

In [ ]:
df[df.columns[(df.isnull().sum()/len(df))>=0.4]].shape

(1345310, 58)

In [ ]:
#removing columns with null values > 40%
df=df[df.columns[(df.isnull().sum()/len(df))<0.4]]
df.shape

(1345310, 93)

 **Creating 2 different excel workbooks for 2 types of variables:**<br>
**1. Variables with missing values** <br>
**2. Variables with no missing values** <br>

**Each workbook has has seperate sheets for all the variables. Each sheet will have the following statistical information for the variable when grouped by our target variable loan_status: count, mean, median, standard deviation, min, 0th percentile, 25th percentile, 50th percentile, 50th percentile, 75th percentile, 100th percentile, max and missing value%**

In [ ]:
files.upload()  # This will prompt to upload the analysis_functions_columns.py file from your local machine

# Assuming the analysis_functions_columns.py file is uploaded to /content directory
shutil.move("analysis_functions_columns.py", "/content/analysis_functions_columns.py")

# Import custom functions from analysis_functions_columns.py
from analysis_functions_columns import analyze_columns_with_missing_values_by_loan_status, analyze_columns_by_loan_status, export_report_to_excel

# Generate the descriptive statistics report for columns with missing values
results_missing_values = analyze_columns_with_missing_values_by_loan_status(df)
export_report_to_excel(results_missing_values, '/content/drive/MyDrive/Lending_Club_Analysis/Output_files/statistics_columns_with_missing_values.xlsx')

# Generate the descriptive statistics report report for columns with no missing values
results_no_missing_values = analyze_columns_by_loan_status(df)
export_report_to_excel(results_no_missing_values, '/content/drive/MyDrive/Lending_Club_Analysis/Output_files/complete_columns_statistics.xlsx')

Saving analysis_functions_columns.py to analysis_functions_columns.py
Report exported successfully to /content/drive/MyDrive/Lending_Club_Analysis/Output_files/statistics_columns_with_missing_values.xlsx
Report exported successfully to /content/drive/MyDrive/Lending_Club_Analysis/Output_files/complete_columns_statistics.xlsx


In [ ]:
#Upload the files with relevant columns for EDA

# Define the path to the Google Drive folder
drive_path = '/content/drive/MyDrive/Lending_Club_Analysis/Output_files/'

# Export the DataFrame to a CSV file
df.to_csv(drive_path + 'dataframe_for_eda.csv', index=False)

print("DataFrame exported successfully to CSV.")

DataFrame exported successfully to CSV.
